# Практика
Используемые библиотеки

In [52]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [ ]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('UTF8')))
#df = pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

In [ ]:
def func(res, norm):
    try:
        float_res = float(res)
    except Exception:
        return 'Ошибка формата результата анализа'
    nums = [float(x.replace(',', '.')) for x in re.findall(r"[+]?(?:\d*\,\d+|\d+)", norm.strip())]
    result = float_res < nums[-1] if norm.startswith('не более') else\
        (nums[-2] < float(res) < nums[-1] if norm.startswith('в пределах') else False)
    return 'Норматив выполнен' if result else 'Норматив не выполнен'


new_col_name = 'Вывод'
df[new_col_name] = df.apply(lambda x: func(x['Результат анализа'], x['Норматив']), axis=1)
df.set_index('Показатель')
df

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [ ]:
fruit = ['O', 'A']  #варианты фруктов


#Функция выбирает случайным образом 3 фрукта из ящика. Для ускорения можно было бы сразу завершить эксперимент как неуспешный, если попалось яблоко.
def random_choice():
    n_orange = 5
    n_apple = 4
    result_choice = []
    for i in range(0, 3):
        p_orange = n_orange / (n_apple + n_orange)
        choice = np.random.choice(fruit, 1, p=[p_orange, 1 - p_orange])[0]
        result_choice.append(choice)
        if choice == 'O':
            n_orange -= 1
        else:
            n_apple -= 1
    return result_choice


N = 100000  # число экспериментов с выбором 3 фруктов из ящика
N_success = 0  #число успешных исходов эксперимента

for i in range(N):
    choice = random_choice()
    if all(x == 'O' for x in choice): N_success += 1

probability = N_success / N  #искомая вероятность
probability

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
details = ['S', 'NS']  #варианты деталей
n_s = 7
n_ns = 3


#Функция осуществляет выбор деталей до первой стандартной. Если в результате первой проверки получили S, то сразу завершаем эксперимент,
# как неуспешный, в противном случае делаем еще одну проверку. Возвращаем "успех", если получили S во 2-м эксперименте
def random_choice_():
    p_s = n_s / (n_s + n_ns)
    choice = np.random.choice(details, 1, p=[p_s, 1 - p_s])[0]
    if (choice == details[0]): return False
    p_s = n_s / (n_s + n_ns - 1)  #обновили верояность выбора стандартной детали
    choice = np.random.choice(details, 1, p=[p_s, 1 - p_s])[0]
    return True if choice == details[0] else False


N = 100000  # число экспериментов с проверкой деталей
N_success = 0  #число успешных исходов эксперимента

for i in range(N):
    N_success += random_choice_()

probability = N_success / N  #искомая вероятность
probability